In [44]:
import pandas as pd
import numpy as np
import time
import re
import requests
import pickle
import joblib

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from eunjeon import Mecab
from keras.models import load_model
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from datetime import datetime

In [53]:
재무제표 = pd.read_csv('기업재무제표1.csv', encoding='cp949')

column_list = 재무제표.columns[3:]

for i in np.arange(0,len(column_list),2):
    for j in range(재무제표.shape[0]):
        if np.isnan(재무제표[column_list[i]][j]) :
            재무제표[column_list[i]][j]=재무제표[column_list[i+1]][j]

재무제표=재무제표.drop(재무제표.columns[np.arange(4, (len(재무제표.columns)+1),2)],axis=1)

columns_ = ['회사명', '거래소코드', '회계년도', '자산', '영업이익', '부채', '자본', '매출액', '유동자산', '유동부채', '이익잉여금']
재무제표.columns = columns_

In [54]:
재무제표['운전자본'] = 재무제표['유동자산']-재무제표['유동부채']
재무제표['자기자본'] = 재무제표['자산']-재무제표['부채']

재무제표['운전자산총자본비율'] = (재무제표['운전자본'] / 재무제표['자산']) * 100
재무제표['이익잉여금총자산비율'] = (재무제표['이익잉여금'] / 재무제표['자산']) * 100
재무제표['총자산이익율'] = (재무제표['영업이익'] / 재무제표['자산']) * 100
재무제표['시장가부채비율'] = (재무제표['자산'] / 재무제표['부채']) * 100
재무제표['매출액회전율'] = (재무제표['매출액'] / 재무제표['자산']) * 100

In [55]:
사용할재무비율 = 재무제표[['회사명', '회계년도', '운전자산총자본비율', '이익잉여금총자산비율', '총자산이익율', '시장가부채비율', '매출액회전율']]
사용할재무비율.회사명 = 사용할재무비율.회사명.str.replace('\(주\)', '')
사용할재무비율

C:\Users\dbswo\AppData\Local\Temp\ipykernel_5748\4262943400.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  사용할재무비율.회사명 = 사용할재무비율.회사명.str.replace('\(주\)', '')
C:\Users\dbswo\AppData\Local\Temp\ipykernel_5748\4262943400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  사용할재무비율.회사명 = 사용할재무비율.회사명.str.replace('\(주\)', '')


,회사명,회계년도,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율
0,인트로메딕,12-20,29.595451,-134.482205,-11.306465,357.049925,19.087055
1,넵튠,12-20,6.990555,13.825347,-0.603778,306.267660,12.179336


In [6]:
dif_3m = relativedelta(months=3)
dif_12m = relativedelta(months=12)

오늘날짜 = datetime.now().date()

기사수집종료날짜 = 오늘날짜 - dif_3m
기사수집시작날짜 = 기사수집종료날짜 - dif_12m

종료연도 = str(기사수집종료날짜.year)
종료월 = str(기사수집종료날짜.month)
종료일 = str(기사수집종료날짜.day)

시작연도 = str(기사수집시작날짜.year)
시작월 = str(기사수집시작날짜.month)
시작일 = str(기사수집시작날짜.day)

In [7]:
print(기사수집종료날짜, 기사수집시작날짜)

2022-04-05 2021-04-05


In [8]:
column_ = ['기업', '기사발행일', '기사제목', '뉴스기사본문']
뉴스_df = pd.DataFrame(columns=column_)

driver = webdriver.Chrome()

start = (시작연도 + '.0' + 시작월 + '.0' + 시작일)
end = (종료연도 + '.0' + 종료월 + '.0' + 종료일)
start_= (시작연도 + '0' + 시작월 + '0' + 시작일)
end_ = (종료연도 + '0' + 종료월 + '0' + 종료일)

for 기업 in  사용할재무비율.회사명:

    # 나중에 뉴스_df와 concat할 임시 df 생성 및 임시 리스트들 생성
    column_ = ['기업', '기사발행일', '기사제목', '뉴스기사본문']
    임시_df = pd.DataFrame(columns=column_)    

    # 임시_df에 들어갈 리스트 생성
    본문리스트 = []
    날짜리스트 = []
    제목리스트 = []
    기업이름 = []

    # while 종료 조건으로 쓸 리스트 생성
    newslist = []
    datelist = []
    
    page = 1

    # 페이지수가 나와있지않으므로 맨끝에 page에 10씩더해서 계속 다음페이지로 이동
    while page <200:

        url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query='+'"'+기업+'"'+'&sort=0&photo=0&field=0&pd=3&ds='+start+'&de='+end+'&cluster_rank=19&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from'+start_+'to'+end_+',a:all&start='+str(page)
        driver.get(url)

        response = requests.get(url)
        soup = BeautifulSoup(response.text, "lxml")

        # a태그중에서 class가 info인 것과 span태그에 class가 info 인 것 가져옴 
        news_titles = soup.select("a.info")
        dates = soup.select('span.info')

        # 네이버기사와 신문사기사 둘다 하이퍼링크가 있을경우는 기사당 news가 2개씩 네이버 기사가 없을 경우는 1개씩만 링크가 추출된다
        # 어림잡아 news_titles수가 10개 미만일 경우는 기사의 수가 5개에서 10개 미만이라는 뜻으로 해석하여 10개 이상만 링크와 기사날짜를 추출
        if len(news_titles) >= 10:

            for news in news_titles:
                title = news.attrs['href']
                newslist.append(title)

            for date in dates:
                news_date = date.text
                datelist.append(news_date)

            # 네이버 검색시 page number가 다음페이지로 갈때마다 1, 11, 21 이렇게 10씩 더해지는데 다음페이지가 없을경우 
            # 마지막 기사만을 포함한 같은 페이지를 계속 반환함. 따라서 기사와 날짜 둘 다 중복되서 저장될경우 종료함
            if (newslist[-1]==newslist[-2]) & (datelist[-1]==datelist[-2]):
                break

            page += 10

            time.sleep(1)
        
        # 기사수 5개 미만시 break, 기사없는기업 리스트에 저장함
        else:
            
            break
    
    # sid=101은 네이버 경제기사를 의미하는 듯함. 경제기사만 추출
    newslist = [news for news in newslist if 'sid=101' in news]
    
    # news_titles 뉴스기사 url 리스트가 존재시
    if news_titles:

        # 뉴스기사 url 자체에서는 text가 안가져와지는 특이사항 발생, 찾아보니 네이버기사는 인터넷에 떠야 페이지가 작동하는? 방식이라함
        # 셀레니움을 통해서 뉴스기사 url 주소로 창을 띄움
        for news in newslist:
            url = news
            driver.get(url)
            
            # 뉴스기사 url에서 본문과 제목, 기사작성날짜를 리스트에 저장함
            # 데이터프레임에 직접 행을 지정해주기는 번거로움..
            try:

                날짜 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
                날짜리스트.append(날짜)

                제목 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[2]/h2').text
                제목리스트.append(제목)

                본문 = driver.find_element('xpath', '//*[@id="dic_area"]').text
                본문리스트.append(본문)
                time.sleep(1)

                기업이름.append(기업)
            
            ## 로딩이 안되서 데이터를 못가져올 경우를 대비해 sleep 3초 주고 다시 시도
            except:

                time.sleep(3)

                try:

                    날짜 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span').text
                    날짜리스트.append(날짜)

                    제목 = driver.find_element('xpath', '//*[@id="ct"]/div[1]/div[2]/h2').text
                    제목리스트.append(제목)

                    본문 = driver.find_element('xpath', '//*[@id="dic_area"]').text
                    본문리스트.append(본문)
                    time.sleep(1)

                    기업이름.append(기업)
                
                # 그래도 데이터를 가져오지 못하는 경우는 페이지에 문제가 있다고 판단하여 PASS
                except:

                    pass

    # 혹시나 뉴스기사 url 리스트가 없을 경우는 pass                
    else:
        pass

    # 기업별로 가져온 날짜와 본문, 제목, 기업이름을 임시 데이터프레임으로 저장    
    임시_df.기사발행일 = 날짜리스트
    임시_df.뉴스기사본문 = 본문리스트
    임시_df.기사제목 = 제목리스트
    임시_df.기업 = 기업이름
    
    # 임시 데이터프레임을 뉴스 데이터프레임에 아래로 결합
    뉴스_df = pd.concat([뉴스_df, 임시_df])

# 앞에서 while 종료 조건이 같은 기사 2번저장인데 이럴 경우 중복으로 저장이 되야 종료되기때문에
# 중복기사 행 제거
뉴스_df.drop_duplicates(inplace=True)


In [9]:
뉴스_df

,기업,기사발행일,기사제목,뉴스기사본문
0,인트로메딕,2022.04.03. 오후 5:27,"쌍용차·에디슨EV·인트로메딕 등 42개社, 무더기 상폐 위기","유가증권 4개社\n코스닥 38개社 대상\n\n쌍용차, 2년 연속\n감사의견 '거절'..."
1,인트로메딕,2022.04.04. 오전 6:02,당신 계좌에 이 종목 있다면 긴장하라…‘상폐 위기’ 42종목 어디?,유가증권 기업 4곳·코스닥38곳 감사의견 ‘부적정’···상폐위기\n에디슨 EV·인트...
2,인트로메딕,2022.03.28. 오후 7:08,"거래소, 인트로메딕 주권매매거래정지 기간변경",[이데일리 양지윤 기자] 한국거래소 코스닥시장본부는 28일 인트로메딕(150840)...
3,인트로메딕,2022.03.14. 오전 11:32,"거래소 ""인트로메딕, 감사의견 비적정 내일까지 확인…거래정지""",[서울=뉴시스] 이승주 기자 = 한국거래소 코스닥시장본부는 14일 인트로메딕의 감사...
4,인트로메딕,2022.03.14. 오후 6:04,"인트로메딕, 감사범위제한 의견거절…상폐사유 발생",[이데일리 안혜신 기자] 한국거래소 코스닥시장본부는 인트로메딕(150840)에 대해...
5,인트로메딕,2021.12.14. 오후 4:49,"인트로메딕, 20일간 거래정지...이유는",기사내용 요약\n'80% 감자'·'5분의 1 액분' 진행…7~28일 거래 정지\n회...
6,인트로메딕,2021.12.06. 오후 12:06,"인트로메딕, 싸이월드Z·경기도자원봉사센터와 협약 체결",[이데일리 김인경 기자] 영상 진단 의료기기 전문기업 인트로메딕(150840)이 싸...
7,인트로메딕,2021.12.24. 오후 5:14,"거래소, 인트로메딕 주권매매거래정지 해제",한국거래소 코스닥시장본부는 인트로메딕의 감자 주권 및 액면분할 변경상장에 따라 주권...
8,인트로메딕,2021.12.02. 오후 4:32,"인트로메딕, 오는 7일부터 주권매매거래 정지",[이데일리 이은정 기자] 한국거래소 코스닥시장본부는 인트로메딕(150840)에 대해...
9,인트로메딕,2021.11.29. 오후 4:10,"인트로메딕, 200억원 규모 전환사채 발행 결정","[이데일리 김소연 기자] 인트로메딕(150840)은 타법인 증권 취득 자금, 기타자..."


In [10]:
제거할기사제목 = ['증시 일정', '증시일정', '장마감후', '장 마감 후', '장마감공시', '증시 캘린더', '재송', '투자정보', \
    '코스닥 기업공시', '장중 주요', '코스닥 3분기 결산', '주요 뉴스 및 공시', '기업공시',' 기업 공시', '장 종료 후', \
        '주요 정보', '오전 공시', '투자정보', '기업 공시', '오늘의 메모', '재테크 캘린더', '추천주 정리', '희망복원 주식클리닉', \
        '\d{1,2}월 \d{1,2}일', '오늘의 주요 공시', '코스닥 공시', '코스닥 메모', '대박 공모주', '<표>', '오늘의 리포트', \
        '주식상담소', '주식왕 따라잡기', '주식컨설팅', '\d{4} 증시', '춤추는 테마주', '개장시황', \
        '폭등신호 터진', '대폭등', '매드머니', '주담과 Q&A', '굿바이 \d{4}', '지금 당장 사라', '폭등주', \
        '\d{4}%', '김정일 사망', '종목대탐험', '종목신호등', 'VIP CLUB', '수급유망주', '기상도', '중소형주', \
        '국민주식고충처리반', '유망주', '머니Q', '시초가잡아라', '기관 Q&A', '부동산에 투자하려면', '베스트애널리스트', \
        '코스닥협회장 취임', '코스닥협회', '부동산에 투자하는 방법', '국가품질 경영대회', '증시일정', '머니Q', '추천종목', '티타임 공략주', \
        '수급유망주', '종목배틀', '\d{4} 증시 결산', '기업설명회', '종목신호등', 'VIP CLUB 추천주', '내일장 공략주 10선', '주간컨센서스동향', \
        '주담과 Q&A', '조회공시', '주가급등 사유', '관련株들', '게임株', '이시각 Up&Down', '장내 매도', '증시기상도', \
        '마감시황', '주식부자 속출', '티타임 공략주', '\d{1,2}일 증권사 추천종목', '기업IR소식']
제거제목리스트 = '|'.join(제거할기사제목)

In [11]:
뉴스중복제거 = 뉴스_df.drop_duplicates(['뉴스기사본문'])
뉴스 = 뉴스중복제거[~뉴스중복제거['기사제목'].str.contains(제거제목리스트, na=False, case=False)]

pattern1 = r'\([^)]*\)'
pattern2 = r"\[([^]]+)\]"

뉴스['뉴스기사본문전처리'] = [re.sub(pattern1, '', s) for s in 뉴스['뉴스기사본문']]
뉴스['뉴스기사본문전처리'] = [re.sub(pattern2, '', s) for s in 뉴스['뉴스기사본문전처리']]
뉴스['뉴스기사본문전처리'] = [re.sub('[^/^$|\s+/가-힣\.]', '', s) for s in 뉴스['뉴스기사본문전처리']]
뉴스['뉴스기사본문전처리'] = [re.sub('[[ \s]{2,}\.{2,}]', '', s) for s in 뉴스['뉴스기사본문전처리']]
뉴스['뉴스기사본문전처리'] = [re.sub('\n', '', s) for s in 뉴스['뉴스기사본문전처리']]
뉴스['뉴스기사본문전처리'] = [re.sub('[/+]', '', s) for s in 뉴스['뉴스기사본문전처리']]
뉴스['뉴스기사본문전처리'] = [re.sub('\r', '', s) for s in 뉴스['뉴스기사본문전처리']]
뉴스.drop('뉴스기사본문', axis=1, inplace=True)
뉴스.reset_index(drop=True, inplace=True)

C:\Users\dbswo\AppData\Local\Temp\ipykernel_5748\500996306.py:10: FutureWarning: Possible nested set at position 1
  뉴스['뉴스기사본문전처리'] = [re.sub('[[ \s]{2,}\.{2,}]', '', s) for s in 뉴스['뉴스기사본문전처리']]


In [12]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

mecab = Mecab()
# Mecab으로 토큰화
뉴스['뉴스기사본문전처리'] = 뉴스['뉴스기사본문전처리'].apply(lambda x: mecab.morphs(x))
# 한 글자 단어 제거
뉴스['뉴스기사본문전처리'] = 뉴스['뉴스기사본문전처리'].apply(lambda x: [word for word in x if len(word) > 1])
# stopwords에 있는 불용어 제거
뉴스['뉴스기사본문전처리'] = 뉴스['뉴스기사본문전처리'].apply(lambda x: [word for word in x if word not in stopwords])

In [13]:
# loading
with open('./model/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

뉴스기사본문전처리 = np.array(뉴스['뉴스기사본문전처리'])

뉴스기사본문전처리 = tokenizer.texts_to_sequences(뉴스기사본문전처리)
뉴스기사본문전처리 = sequence.pad_sequences(뉴스기사본문전처리, maxlen=100)


In [14]:
뉴스기사본문전처리

array([[   0,    0,    0, ..., 2761,   71,  879],
       [ 168,   46,   56, ...,  392,   17,   17],
       [   0,    0,    0, ...,  163,  180,   71],
       ...,
       [   0,    0,    0, ..., 4244, 1773,  156],
       [   0,    0,    0, ..., 3215,   34,   47],
       [ 598,    2, 2177, ...,  112,  105,  116]])

In [15]:
model_t = load_model("./model/02-0.170406.hdf5")

predict = model_t.predict(뉴스기사본문전처리)
predict = predict.round()

1/1 [==============================] - 0s 403ms/step


In [16]:
뉴스['부도기사분류'] = predict
뉴스

,기업,기사발행일,기사제목,뉴스기사본문전처리,부도기사분류
0,인트로메딕,2022.04.03. 오후 5:27,"쌍용차·에디슨EV·인트로메딕 등 42개社, 무더기 상폐 위기","[유가증권, 코스닥, 대상, 쌍용, 연속, 감사, 의견, 거절, 자동차, 선도, 전...",1.0
1,인트로메딕,2022.04.04. 오전 6:02,당신 계좌에 이 종목 있다면 긴장하라…‘상폐 위기’ 42종목 어디?,"[유가증권, 기업, 코스닥, 감사, 의견, 부적, 정상, 위기, 에디슨, 인트, 로...",1.0
2,인트로메딕,2022.03.28. 오후 7:08,"거래소, 인트로메딕 주권매매거래정지 기간변경","[한국, 거래소, 코스닥, 시장, 본부, 대해, 개선, 기간, 부여, 한다고, 밝혔...",1.0
3,인트로메딕,2022.03.14. 오전 11:32,"거래소 ""인트로메딕, 감사의견 비적정 내일까지 확인…거래정지""","[이승주, 기자, 한국, 거래소, 코스닥, 시장, 본부, 감사, 의견, 적정, 라는...",1.0
4,인트로메딕,2022.03.14. 오후 6:04,"인트로메딕, 감사범위제한 의견거절…상폐사유 발생","[한국, 거래소, 코스닥, 시장, 본부, 인트, 로메, 대해, 감사, 범위, 제한,...",1.0
5,인트로메딕,2021.12.14. 오후 4:49,"인트로메딕, 20일간 거래정지...이유는","[기사, 내용, 요약, 감자, 진행, 거래, 정지, 회사, 기술, 특례, 상장, 기...",1.0
6,인트로메딕,2021.12.06. 오후 12:06,"인트로메딕, 싸이월드Z·경기도자원봉사센터와 협약 체결","[영상, 진단, 의료, 기기, 전문, 기업, 싸이월드, 제트, 경기도, 자원, 봉사...",1.0
7,인트로메딕,2021.12.24. 오후 5:14,"거래소, 인트로메딕 주권매매거래정지 해제","[한국, 거래소, 코스닥, 시장, 본부, 인트, 로메, 감자, 주권, 액면, 분할,...",0.0
8,인트로메딕,2021.12.02. 오후 4:32,"인트로메딕, 오는 7일부터 주권매매거래 정지","[한국, 거래소, 코스닥, 시장, 본부, 인트, 로메, 대해, 주식, 병합, 분할,...",0.0
9,인트로메딕,2021.11.29. 오후 4:10,"인트로메딕, 200억원 규모 전환사채 발행 결정","[인트, 로메, 법인, 증권, 취득, 자금, 기타, 자금, 확보, 위해, 규모, 전...",0.0


In [18]:
뉴스

,기업,기사발행일,기사제목,뉴스기사본문전처리,부도기사분류
0,인트로메딕,2022.04.03. 오후 5:27,"쌍용차·에디슨EV·인트로메딕 등 42개社, 무더기 상폐 위기","[유가증권, 코스닥, 대상, 쌍용, 연속, 감사, 의견, 거절, 자동차, 선도, 전...",1.0
1,인트로메딕,2022.04.04. 오전 6:02,당신 계좌에 이 종목 있다면 긴장하라…‘상폐 위기’ 42종목 어디?,"[유가증권, 기업, 코스닥, 감사, 의견, 부적, 정상, 위기, 에디슨, 인트, 로...",1.0
2,인트로메딕,2022.03.28. 오후 7:08,"거래소, 인트로메딕 주권매매거래정지 기간변경","[한국, 거래소, 코스닥, 시장, 본부, 대해, 개선, 기간, 부여, 한다고, 밝혔...",1.0
3,인트로메딕,2022.03.14. 오전 11:32,"거래소 ""인트로메딕, 감사의견 비적정 내일까지 확인…거래정지""","[이승주, 기자, 한국, 거래소, 코스닥, 시장, 본부, 감사, 의견, 적정, 라는...",1.0
4,인트로메딕,2022.03.14. 오후 6:04,"인트로메딕, 감사범위제한 의견거절…상폐사유 발생","[한국, 거래소, 코스닥, 시장, 본부, 인트, 로메, 대해, 감사, 범위, 제한,...",1.0
5,인트로메딕,2021.12.14. 오후 4:49,"인트로메딕, 20일간 거래정지...이유는","[기사, 내용, 요약, 감자, 진행, 거래, 정지, 회사, 기술, 특례, 상장, 기...",1.0
6,인트로메딕,2021.12.06. 오후 12:06,"인트로메딕, 싸이월드Z·경기도자원봉사센터와 협약 체결","[영상, 진단, 의료, 기기, 전문, 기업, 싸이월드, 제트, 경기도, 자원, 봉사...",1.0
7,인트로메딕,2021.12.24. 오후 5:14,"거래소, 인트로메딕 주권매매거래정지 해제","[한국, 거래소, 코스닥, 시장, 본부, 인트, 로메, 감자, 주권, 액면, 분할,...",0.0
8,인트로메딕,2021.12.02. 오후 4:32,"인트로메딕, 오는 7일부터 주권매매거래 정지","[한국, 거래소, 코스닥, 시장, 본부, 인트, 로메, 대해, 주식, 병합, 분할,...",0.0
9,인트로메딕,2021.11.29. 오후 4:10,"인트로메딕, 200억원 규모 전환사채 발행 결정","[인트, 로메, 법인, 증권, 취득, 자금, 기타, 자금, 확보, 위해, 규모, 전...",0.0


In [62]:
전체기사수 = pd.DataFrame(뉴스.groupby(['기업']).count()['부도기사분류'])
전체기사수.reset_index(drop = False, inplace = True)
전체기사수.columns = ['회사명', '전체기사수']

부도기사수 = pd.DataFrame(뉴스[뉴스.부도기사분류==1].groupby(['기업']).count()['부도기사분류'])
부도기사수.reset_index(drop = False, inplace = True)
부도기사수.columns = ['회사명', '부도기사수']

부도기사비율 = pd.merge(전체기사수, 부도기사수, on='회사명')
부도기사비율['부도기사비율'] = (부도기사비율['부도기사수'] / 부도기사비율['전체기사수']) * 100
부도기사비율.drop(['전체기사수', '부도기사수'], axis=1, inplace=True)
부도기사비율
# 부도기사비율 = 부도기사수['부도기사수'] / 전체기사수['전체기사수']
# 부도기사비율.fillna(0, inplace=True)
# 부도기사비율

,회사명,부도기사비율
0,넵튠,8.333333
1,인트로메딕,43.750000


In [63]:
사용할재무비율_  = pd.merge(사용할재무비율, 부도기사비율, on='회사명', how='left')
사용할재무비율_

,회사명,회계년도,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,부도기사비율
0,인트로메딕,12-20,29.595451,-134.482205,-11.306465,357.049925,19.087055,43.750000
1,넵튠,12-20,6.990555,13.825347,-0.603778,306.267660,12.179336,8.333333


In [60]:
ml_model = joblib.load('./model/svc_model.pkl')
robscaler = joblib.load('./model/robustscaler.pkl')

In [65]:
predict = ml_model.predict(사용할재무비율_.iloc[:,2:])
predict

c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


array([1, 0], dtype=int64)

In [39]:
사용할재무비율_.iloc[:,2:]

,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,부도기사비율
0,29.595451,-134.482205,-11.306465,357.049925,19.087055,43.750000
1,6.990555,13.825347,-0.603778,306.267660,12.179336,8.333333


In [64]:
사용할재무비율_.iloc[:,2:] = robscaler.transform(사용할재무비율_.iloc[:,2:])
사용할재무비율_.iloc[:,2:]

c:\Users\dbswo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- 부도기사비율
- 운전자산총자본비율
Feature names seen at fit time, yet now missing:
- 부도기사 비율
- 운전자본비율

  warnings.warn(message, FutureWarning)


,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,부도기사비율
0,0.411519,-2.109706,-0.760272,0.678155,-0.843133,1.622862
1,-0.102400,0.184311,-0.082728,0.447989,-0.968081,0.028953


In [44]:
사용할재무비율.iloc[:,2:]

,운전자산총자본비율,이익잉여금총자산비율,총자산이익율,시장가부채비율,매출액회전율,부도기사비율
0,29.595451,-134.482205,-11.306465,357.049925,19.087055,0.437500
1,6.990555,13.825347,-0.603778,306.267660,12.179336,0.083333


In [43]:
predict

array([0, 0], dtype=int64)